1. Run codes：<br>
   1.1 运行性：经本地运行后，发现代码能够顺利运行；<br>
   1.2 复现性：代码中train_test_split(random_state=111)、KFold(shuffle=True, random_state=111)固定了随机种子，确保数据划分和交叉验证的随机性可控。通过改变models_config字典中的函数（将Ridge改为OLS、LASSO、ElasticNet）并调整对应参数，可完成原结果复现并实现问题要求。

2. Data processsing: <br>
   X_scaled 是否存在Inf: False <br>
   X_scaled 是否存在NaN: False <br>
   运行代码，检查数据处理结果，发现数据已正确处理，可以用于模型训练。


3. Sample choice：<br>
   3.1 划分样本：代码正确地将样本集划分为训练集和测试集；<br>
   3.2 数据泄露：由于处理缺失值时使用全部的样本，因此可能存在数据泄露问题。


4. Suggstions <br>
    4.1 扩大超参数搜索范围以提升模型性能 <br>
    在代码的模型配置部分（models_config），超参数的搜索范围设置过窄（例如 Ridge 的 alpha 仅尝试 [3000, 4500,5000]），这可能导致模型无法找到最优参数组合。<br>
    且窄范围的超参数搜索可能导致模型陷入局部最优，无法充分发挥正则化模型的潜力，尤其是在特征较多（25个特征）时。<br>
    对正则化模型（如 Lasso、Ridge、ElasticNet）的 alpha 参数，可以使用对数刻度的范围（覆盖多个数量级），而非单一值。<br>
    具体可参考以下代码：



    4.2 将一些特征作为分类变量处理 <br>
    在代码的特征处理部分中，一些变量被手动赋予了顺序和大小，例如交易权属、房屋年限。 <br>
    尽管手动赋值正确地保留了各个类别对房产价格影响的顺序，但是间距设置不当可能扭曲得到的系数,尤其是对间距数值敏感的线性模型。<br>
    因此对于这些特征使用独热编码可能产生更好的效果。<br>

    4.3 文本处理方法提取更有价值的特征 <br>

    改进建议：使用TF-IDF提取文本特征。  <br>
    当前代码中对文本特征的处理主要是基于关键词匹配，这种方法会丢失很多有价值的信息。建议使用TF-IDF(词频-逆文档频率)来提取文本特征，这能更好地捕捉文本中的重要词汇信息。<br>
    特征工程中的文本特征处理可以更加精细化。当前对文本特征(房屋优势、核心卖点、周边配套、交通出行)的处理相对简单，可以引入更先进的<br>
    代码如下：<br>

    4.4 通过可视化手段选择更为合适的变量 <br>
    在区分分类变量和数据变量之后，可以通过分析各个变量分别与房价的关系，从而选择对房价影响更为显著的变量参与模型。 <br>
    代码如下：

In [ ]:
# 4.3

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def enhanced_text_processing(df, n_components=10):
    
    df['combined_text'] = (
        df['房屋优势'].fillna('') + ' ' + 
        df['核心卖点'].fillna('') + ' ' + 
        df['周边配套'].fillna('') + ' ' + 
        df['交通出行'].fillna('')
    )
    
    
    tfidf = TfidfVectorizer(
        max_features=500,  
        stop_words=['的', '了', '在', '是', '我', '有', '和', '就'],  
        ngram_range=(1, 2)  
    )
    

    tfidf_features = tfidf.fit_transform(df['combined_text'])
    
  
    svd = TruncatedSVD(n_components=n_components, random_state=111)
    reduced_features = svd.fit_transform(tfidf_features)
    

    text_features = pd.DataFrame(
        reduced_features,
        columns=[f'text_feature_{i}' for i in range(n_components)]
    )
    
    
    original_features = process_property_text(df)
    
  
    return pd.concat([original_features, text_features], axis=1)

X_14 = enhanced_text_processing(train, n_components=15)

In [ ]:
# 4.4
# List the variables to plot
variables = ['type', 'laundry_options', 'parking_options']

# Plot the distribution of the categorical variables listed
plt.figure(figsize=(20, 10))

for i in range(3):
    labels = []
    sizes = []
    count = 0
    for k, v in dict(df[variables[i]].value_counts()).items():
        labels.append(f"{k} - {round(((v/df.shape[0])*100), 3)}%")
        sizes.append(v)
        count += 1
        
    plt.subplot(1, 3, i+1)
    patches, texts = plt.pie(sizes, colors=sns.color_palette('tab20b', 12), shadow=False, startangle=120)
    plt.legend(patches, labels, loc='lower center')
    plt.axis('equal')
    plt.title("Distribution of "+variables[i], fontsize=15, y=0.85)
    
plt.suptitle("Data Analysis Of Categorical Variables", fontweight='bold', fontsize=20, y=0.9)
plt.show()